# I. Entendimento do negócio

A. Contexto

A dengue é uma doença viral transmitida principalmente pelo mosquito Aedes aegypti e representa um desafio significativo para a saúde pública. Sua rápida disseminação e impacto na população exigem medidas preventivas e de controle eficazes por parte das autoridades de saúde. Com acesso a diversas informações relacionadas aos casos de dengue, o objetivo é prever o número de casos nos próximos 1500 dias.

B. Desafio

O desafio envolve prever a propagação da dengue com base em fatores como condições climáticas, densidade populacional, saneamento básico e aplicação de medidas de controle do vetor. A interação desses fatores pode variar de acordo com as diferentes regiões geográficas do Brasil.

C. Objetivo

- Prever o número de casos nos próximos 1500 dias. Modelo de Regressão
- Auxiliar as autoridades de saúde a direcionar recursos;
- Priorizando as regiões mais susceptíveis;
- Antecipando a necessidade de medidas preventivas.

D. Expectativa

- Avaliação das submissões será feita com base na Raiz Quadrada do Erro-Médio (RMSE)
- Resultados finais devem ser apresentados em um arquivo CSV contendo as datas e as previsões do número de casos de dengue.

# II. Entendimento dos dados

## 0. IMPORTS

### 0.1. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from unidecode import unidecode
import inflection
import datetime
import math
import random

import xgboost as xgb

from sklearn               import ensemble as en
from sklearn               import preprocessing as pp
from sklearn.metrics       import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso

### 0.2. Loading data

In [2]:
tb_fato = pd.read_csv('datasets/tb_fato.csv', low_memory=False, decimal='.')

tb_casos_dengue = pd.read_csv('datasets/tb_casos_dengue.csv', low_memory=False, decimal='.')
tb_agua_parada = agua_parada = pd.read_csv('datasets/tb_agua_parada.csv', low_memory=False, decimal='.')
tb_condicoes_climaticas = pd.read_csv('datasets/tb_condicoes_climaticas.csv', low_memory=False, decimal='.')
tb_falta_higiene = pd.read_csv('datasets/tb_falta_higiene.csv', low_memory=False, decimal='.')

tb_conscientizacao = pd.read_csv('datasets/tb_conscientizacao.csv', low_memory=False, decimal='.')

### 0.3. Helper functions

In [3]:
def rename_columns(cols):
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(' ', '')
    remove_accents = lambda x: unidecode(x)
    cols = list(map(title, cols))
    cols = list(map(spaces, cols))
    cols = list(map(remove_accents, cols))
    cols = list(map(snakecase, cols))
    return cols

def definir_proliferacao(semana):
    if 22 <= semana <= 39:
        return 'baixa'
    elif semana < 15 or semana > 47:
        return 'media'
    elif 15 <= semana < 22 or 39 < semana <= 47:
        return 'alta'
    
## Função de calcular media das semanas
def preencher_media_semana(df, coluna):
    # Loop ao longo das semanas
    for semana in range(1, 53):
        # Calcular a media da coluna para a semana atual
        media_da_semana = df.loc[df['semana_do_ano'] == semana, coluna].mean()

        # Preencher os valores ausentes na coluna para a semana atual com a média calculada
        df.loc[df['semana_do_ano'] == semana, coluna] = df.loc[df['semana_do_ano'] == semana, coluna].fillna(media_da_semana)
    return df2

# Função das métricas de performance dos modelos de machine learning
def ml_error(model_name, y, yhat):
    mae = mean_absolute_error(y, yhat)
    mape = mean_absolute_percentage_error (y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    return pd.DataFrame({'Model Name': model_name,
                         'MAE': mae,
                         'MAPE': mape,
                         'RMSE': rmse}, index=[0])

# Função cross-validation de performance dos modelos de machine learning
def cross_validation(x_training, kfold, model_name, model, verbose=False):
    mae_list = []
    mape_list = []
    rmse_list = []

    for k in reversed(range(1, kfold+1)):
        if verbose:
            print('KFold Number: {}'.format(k))

        # Data inicial e fim para validação
        validation_start_date = x_training['date'].max() - datetime.timedelta(days=k*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta(days=(k-1)*6*7)

        # Filtrar dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # Training dataset
        xtraining = training.drop(['date', 'casos_de_dengue'], axis=1)
        ytraining = training['casos_de_dengue']

        # Validation dataset
        xvalidation = validation.drop(['date', 'casos_de_dengue'], axis=1)
        yvalidation = validation['casos_de_dengue']

        # Model
        m = model.fit(xtraining, ytraining)

        # Prediction
        yhat = m.predict(xvalidation)

        # Performance
        m_result = ml_error(model_name, np.expm1( yvalidation ), np.expm1( yhat ))

        # Guarda a performance de cada iteração kfold
        mae_list.append(m_result['MAE'])
        mape_list.append(m_result['MAPE'])
        rmse_list.append(m_result['RMSE'])
    
    return pd.DataFrame({'Model  Name': model_name,
                         'MAE CV': np.round(np.mean(mae_list), 2).astype(str) + ' +/- ' + np.round(np.std(mae_list), 2).astype(str),
                         'MAPE CV': np.round(np.mean(mape_list), 2).astype(str) + ' +/- ' + np.round(np.std(mape_list), 2).astype(str),
                         'RMSE CV': np.round(np.mean(rmse_list), 2).astype(str) + ' +/- ' + np.round(np.std(rmse_list), 2).astype(str)}, index=[0])

### 0.4. Dicionário de dados

Arquivos


features.csv

Feature	| Description
-- | --
Date 	| identificador único da loja;


## 1. DATA DESCRIPTION

### 1.1. Unir tabelas e renomear colunas

In [4]:
# Unir tabelas
df = pd.merge(tb_fato, tb_casos_dengue, how='left', on='Date', suffixes=('_fato', '_casos'))
df = df.merge(tb_agua_parada, how='left', on='Date', suffixes=('', '_agua'))
df = df.merge(tb_condicoes_climaticas, how='left', on='Date', suffixes=('', '_cli'))
df = df.merge(tb_falta_higiene, how='left', on='Date', suffixes=('', '_hig'))
df = df.merge(tb_conscientizacao, how='left', left_on='conscientização da população', right_on='id', suffixes=('', '_consc'))

df1 = df.copy()

In [5]:
# Renomear colunas
df1.columns = rename_columns(df1.columns)

### 1.2. Data Dimensions

In [6]:
print('Quantidade de linhas: {:,}'.format(df1.shape[0]))
print('Quantidade de colunas: {:,}'.format(df1.shape[1]))

Quantidade de linhas: 27,028
Quantidade de colunas: 22


### 1.3. Data Types

In [7]:
df1.dtypes

date                                           object
conscientizacao_da_populacao                    int64
condicoes_de_moradia_precarias                 object
falta_de_acesso_a_servicos_de_saude            object
migracao_de_pessoas_de_areas_endemicas         object
transporte_de_mercadorias_em_areas_urbanas     object
outros                                         object
presenca_do_mosquito                           object
casos_de_dengue                               float64
presenca_de_criadouros                         object
agua_parada_em_terrenos_baldios                object
presenca_de_piscinas_sem_manutencao            object
presenca_de_recipientes_sem_tampas             object
precipitacao                                   object
umidade                                        object
temperatura                                    object
falta_de_coleta_de_lixo                        object
areas_com_acumulo_de_entulhos                  object
falta_de_controle_de_pragas 

### 1.4. Check NA's

In [8]:
df1.isna().sum()

date                                             0
conscientizacao_da_populacao                     0
condicoes_de_moradia_precarias                 144
falta_de_acesso_a_servicos_de_saude            198
migracao_de_pessoas_de_areas_endemicas         156
transporte_de_mercadorias_em_areas_urbanas     146
outros                                         124
presenca_do_mosquito                           122
casos_de_dengue                               3000
presenca_de_criadouros                         156
agua_parada_em_terrenos_baldios                146
presenca_de_piscinas_sem_manutencao            126
presenca_de_recipientes_sem_tampas             154
precipitacao                                   118
umidade                                        144
temperatura                                    118
falta_de_coleta_de_lixo                        132
areas_com_acumulo_de_entulhos                  126
falta_de_controle_de_pragas                    148
taxa_de_tratamento_de_esgoto   

### 1.5. Check duplicated

In [9]:
df1.duplicated().sum()

13514

## 2. TRATAMENTO DE DADOS

In [10]:
df2 = df1.copy()

In [11]:
colunas_selecionadas = ['casos_de_dengue', 'date', 'presenca_do_mosquito', 'precipitacao', 'umidade', 'temperatura']
df2 = df2.loc[:, colunas_selecionadas]

### 2.1. Tratamento de duplicatas

In [12]:
df2 = df2.drop_duplicates().reset_index(drop=True)

In [13]:
df2.duplicated().sum()

0

### 2.2. Converter e transformar variáveis

#### 2.2.1. Transformar 'indisponivel' em NA

In [14]:
colunas = ['precipitacao', 
           'umidade',
           'temperatura',
           'presenca_do_mosquito']

# Substituir 'indisponivel' por NaN
for col in colunas:
    df2[col] = df2[col].replace('indisponivel', np.nan)

# Substituir status '<>' por NaN
#df2['status'] = df2['status'].replace('<>', np.nan)

#### 2.2.2. Remover as linhas que não tem casos de dengue 

In [15]:
teste = df2.loc[df2['casos_de_dengue'].isna(), :].reset_index(drop=True)
teste.head()

,casos_de_dengue,date,presenca_do_mosquito,precipitacao,umidade,temperatura
0,NaN,2022-09-09,85338,"2,721541","69,4714597525","27,5437740884"
1,NaN,2019-11-23,61676,"19,603274","68,0779578056","25,3532101942"
2,NaN,2019-11-24,70395,"30,420928","60,8851345050","25,8403117691"
3,NaN,2019-11-25,455030,"52,309753","79,2489200784","27,4500480298"
4,NaN,2019-11-26,137839,"25,596321","76,9084041919","29,1414920945"


In [16]:
casos_faltantes = df2.loc[df2['casos_de_dengue'].isna(), :].index
df2 = df2.drop(casos_faltantes, axis=0).reset_index(drop=True)

#### 2.2.3. Converter variáveis

In [17]:
colunas_float = ['precipitacao',
                 'umidade',
                 'temperatura']

# Substituir ',' por '.'
for col in colunas_float:
    df2[col] = df2[col].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

# Converter colunas_float de object para float
for col in colunas_float:
    df2[col] = df2[col].astype(float)

# Converter date de object para datetime
df2['date'] = pd.to_datetime(df2['date'])

In [18]:
# Criar colunas de dia, mes, ano e semana do ano
#df2['dia'] = df2['date'].dt.day
df2['mes'] = df2['date'].dt.month
#df2['ano'] = df2['date'].dt.year
#df2['semana_do_ano'] = df2['date'].dt.weekofyear
df2['semana_do_ano'] = df2['date'].dt.isocalendar().week

# 'ano_semana': semana do ano, no formato string 'YYYY-WW'
#df2['ano_semana'] = df2['date'].dt.strftime( '%Y-%W' )

#### 2.2.4. Preencher NA's

In [22]:
df2['presenca_do_mosquito'].unique()

array(['874324', '869290', '880145', ..., '369384', '299842', '367044'],
      dtype=object)

In [19]:
presenca_mediana = df2['presenca_do_mosquito'].median()
df2['presenca_do_mosquito'] = df2['presenca_do_mosquito'].fillna(presenca_mediana)

TypeError: Cannot convert ['874324' '869290' '880145' ... '369384' '299842' '367044'] to numeric

In [ ]:
preencher_media_semana(df2, ['umidade', 'temperatura', 'precipitacao'])

In [ ]:
colunas_int = ['casos_de_dengue', 
               'presenca_do_mosquito']

# Converter colunas_int de object para int
for col in colunas_int:
    df2[col] = df2[col].astype(int)

In [ ]:
#temp_media = df2['temperatura'].mean()
#df2['temperatura'] = df2['temperatura'].fillna(temp_media)

#umidade_mediana = df2['umidade'].median()
#df2['umidade'] = df2['umidade'].fillna(umidade_mediana)

#precipitacao_mediana = df2['precipitacao'].median()
#df2['precipitacao'] = df2['precipitacao'].fillna(precipitacao_mediana)

#### 2.2.5. Excluir os  demais NA's

In [ ]:
df2 = df2.dropna().reset_index(drop=True)

In [ ]:
df2.isna().sum()

### 2.4. FEATURE ENGINEERING

#### 2.4.1. Separar data

In [ ]:
df2.head()

#### 2.4.2. Separar em 3 grupos

In [ ]:
#df2['proliferacao'] = df2['semana_do_ano'].apply(lambda x: definir_proliferacao(x))

## 3. EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
df3 = df2.copy()

In [ ]:
df3_num = df3.select_dtypes(include = ['int64', 'float64', 'int32', 'float32'])
df3_cat = df3.select_dtypes(exclude = ['int64', 'float64', 'int32', 'float32'])

### 3.1.  Análise Multivariada

In [ ]:
# presenca_do_mosquito, precipitacao, umidade, temperatura
corr = df3_num.corr(method='pearson')
fig, ax = plt.subplots(figsize=(16, 11))
ax = sns.heatmap(corr, annot=True, ax=ax)

### 3.2. Análise Bivariada

In [ ]:
# Define o estilo "whitegrid" do Seaborn
sns.set_style('whitegrid')

#### 3.2.1. casos_de_dengue x tempo

In [ ]:
# Casos de dengue por mês é uma variável em que a média se distancia muito dos outliers e até do q3 
fig, ax = plt.subplots(2, 1, sharey=True, figsize=(15, 8))
sns.barplot(data=df3, x='mes', y='casos_de_dengue', ax=ax[0])
ax[0].set_title('Casos de dengue por Meses do ano')
ax[0].set_ylabel('Média de casos de dengue por mês')
sns.boxplot(data=df3, x='mes', y='casos_de_dengue', ax=ax[1])
ax[1].set_ylabel('Quantidade de casos de dengue por mês')
plt.show()

In [ ]:
# Casos de dengue por semana_do_ano é uma variável em que a média se distancia muito dos outliers e até do q3 
fig, ax = plt.subplots(2, 1, sharey=True, figsize=(15, 8))
sns.barplot(data=df3, x='semana_do_ano', y='casos_de_dengue', ax=ax[0])
ax[0].set_title('Casos de dengue por Semanas do ano')
ax[0].set_ylabel('Média de casos de dengue por semana do ano')
sns.boxplot(data=df3, x='semana_do_ano', y='casos_de_dengue', ax=ax[1])
ax[1].set_ylabel('Quantidade de casos de dengue por semana do ano')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.barplot(data=df3, x='semana_do_ano', y='casos_de_dengue')
ax.set_xlabel('Mês')
ax.set_ylabel('Casos de dengue')
ax.set_title('')
plt.show()

#### 3.2.1. casos_de_dengue x presenca_do_mosquito

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.scatterplot(data=df3, x='presenca_do_mosquito', y='casos_de_dengue')
ax.set_xlabel('Presença do mosquito')
ax.set_ylabel('Casos de dengue')
ax.set_title('A qtd de casos de dengue cresce a medida que cresce a presença do mosquito')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.scatter(x='presenca_do_mosquito', y='casos_de_dengue', data=df3)
plt.show()

#======== Gráfico Presença de Mosquito e Casos de Dengue x Semana
# Crie a figura e o primeiro eixo y
fig, ax1 = plt.subplots(figsize=(15, 8))
sns.lineplot(x=df3['semana_do_ano'], y=df3['casos_de_dengue'], ax=ax1, color='b', label='Casos de Dengue')

# Criar o segundo eixo y
ax2 = ax1.twinx()
sns.lineplot(x=df3['semana_do_ano'], y=df3['presenca_do_mosquito'], ax=ax2, color='r', label='Presença de Mosquitos')

# Legenda
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# Exibir o gráfico
plt.title('Casos de Dengue e Presença de Mosquitos ao longo das Semanas do Ano')
plt.show()

#### 3.2.2. casos_de_dengue x precipitacao

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.scatterplot(data=df3, x='precipitacao', y='presenca_do_mosquito')
ax.set_xlabel('Precipitação')
ax.set_ylabel('Presença do mosquito')
ax.set_title('')
plt.show()

In [ ]:
# Talvez faça sentido transformar em 3 categorias
fig = plt.figure(figsize=(15, 8))
plt.scatter(x='precipitacao', y='presenca_do_mosquito', data=df3)
plt.show()

#### 3.2.3. casos_de_dengue x umidade

In [ ]:
# Talvez faça sentido transformar em 3 categorias
fig = plt.figure(figsize=(15, 8))
plt.scatter(x='umidade', y='presenca_do_mosquito', data=df3)
plt.show()

#======== Gráfico umidade e Casos de Dengue x Semana
# Crie a figura e o primeiro eixo y
fig, ax1 = plt.subplots(figsize=(15, 8))
sns.lineplot(x=df3['semana_do_ano'], y=df3['presenca_do_mosquito'], ax=ax1, color='b', label='Casos de Dengue')

# Criar o segundo eixo y
ax2 = ax1.twinx()
sns.lineplot(x=df3['semana_do_ano'], y=df3['umidade'], ax=ax2, color='r', label='Umidade')

# Legenda
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# Exibir o gráfico
plt.title('Casos de Dengue e Umidade ao longo das Semanas do Ano')
plt.show()

In [ ]:
#======== Gráfico umidade e Casos de Dengue x Semana
# Crie a figura e o primeiro eixo y
#fig, ax1 = plt.subplots(figsize=(15, 8))
#sns.lineplot(x=df3['semana_do_ano'], y=df3['casos_de_dengue'], ax=ax1, color='b', label='Casos de Dengue')

# Criar o segundo eixo y
#ax2 = ax1.twinx()
#sns.lineplot(x=df3['semana_do_ano'], y=df3['umidade'], ax=ax2, color='r', label='Umidade')

# Legenda
#lines, labels = ax1.get_legend_handles_labels()
#lines2, labels2 = ax2.get_legend_handles_labels()
#ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# Exibir o gráfico
#plt.title('Casos de Dengue e Umidade ao longo das Semanas do Ano')
#plt.show()

#### 3.2.4. casos_de_dengue x temperatura

In [ ]:
# Talvez faça sentido transformar em 4 categorias
fig = plt.figure(figsize=(15, 8))
plt.scatter(x='temperatura', y='presenca_do_mosquito', data=df3)
plt.show()

#======== Gráfico Temperatura e Casos de Dengue x Semana
# Crie a figura e o primeiro eixo y
fig, ax1 = plt.subplots(figsize=(15, 8))
sns.lineplot(x=df3['semana_do_ano'], y=df3['casos_de_dengue'], ax=ax1, color='b', label='Casos de Dengue')

# Criar o segundo eixo y
ax2 = ax1.twinx()
sns.lineplot(x=df3['semana_do_ano'], y=df3['temperatura'], ax=ax2, color='r', label='Temperatura')

# Legenda
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# Exibir o gráfico
plt.title('Casos de Dengue e Temperatura ao longo das semanas do ano')
plt.show()

#### 3.2.5. casos_de_dengue x data

In [ ]:
#fig = plt.figure(figsize=(15, 8))
#sns.lineplot(x=df3['mes'], y=df3['casos_de_dengue'])
#plt.show()

fig = plt.figure(figsize=(15, 8))
sns.lineplot(x=df3['semana_do_ano'], y=df3['casos_de_dengue'])
plt.show()

In [ ]:
#fig = plt.figure(figsize=(15, 8))
#sns.lineplot(x=df3['ano'], y=df3['casos_de_dengue'])
#plt.show()

#### 3.2.6. casos_de_dengue x proliferacao

In [ ]:
df3.head()

In [ ]:
#fig = plt.figure(figsize=(7, 5))
#plt.bar(df3['proliferacao'])
#plt.show

### 3.3. Análise Univariada

#### 3.3.1. Casos de dengue

In [ ]:
q3 = df3['casos_de_dengue'].quantile(0.75)
casos_comuns = df3[df3['casos_de_dengue'] < 41350]

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(x=df3['casos_de_dengue'])
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(df3['casos_de_dengue'], bins=10)
plt.show()

#### 3.3.2. Variáveis numéricas

In [ ]:
# Plotar histogramas para todas as colunas numéricas
df3.hist(figsize=(15, 10))
plt.tight_layout()
plt.show()

#### 3.3.3. Variáveis categóricas

# III. Preparação dos dados

## 4. DATA PREPARATION

In [ ]:
df4 = df3.copy()

In [ ]:
#colunas_selecionadas = ['casos_de_dengue', 'presenca_do_mosquito', 'precipitacao', 'umidade', 'temperatura', 'status']
#df4 = df4.loc[:, colunas_selecionadas]
df4.head()

### 4.1. Normalização do fenômeno

In [ ]:
df4['casos_de_dengue'] = np.log1p(df4['casos_de_dengue'])

In [ ]:
sns.displot(df4['casos_de_dengue']);

### 4.2. Rescala de Variáveis Numéricas

In [ ]:
df3.describe().T

In [ ]:
mm = pp.MinMaxScaler()
ss = pp.StandardScaler()
rs = pp.RobustScaler()

df4['presenca_do_mosquito'] = mm.fit_transform(df4[['presenca_do_mosquito']])
df4['precipitacao']         = ss.fit_transform(df4[['precipitacao']])
df4['umidade']              = ss.fit_transform(df4[['umidade']])
df4['temperatura']          = ss.fit_transform(df4[['temperatura']])

In [ ]:
df4.describe().T

### 4.3. Encodar variáveis categóricas

In [ ]:
#df4 = pd.get_dummies(df4, prefix=['status'], columns=['status'])
#df4 = pd.get_dummies(df4, prefix=['proliferacao'], columns=['proliferacao'])

### 4.4. Transformação de natureza

In [ ]:
# mes
df4['mes_sin'] = df4['mes'].apply( lambda x: np.sin( x * (2 * np.pi/12) ) )
df4['mes_cos'] = df4['mes'].apply( lambda x: np.cos( x * (2 * np.pi/12) ) )

# week_of_year
df4['semana_do_ano_sin'] = df4['semana_do_ano'].apply( lambda x: np.sin( x * (2 * np.pi/52) ) )
df4['semana_do_ano_cos'] = df4['semana_do_ano'].apply( lambda x: np.cos( x * (2 * np.pi/52) ) )

# day_of_week
#df4['day_of_week_sin'] = df4['day_of_week'].apply( lambda x: np.sin( x * (2 * np.pi/7) ) )
#df4['day_of_week_cos'] = df4['day_of_week'].apply( lambda x: np.cos( x * (2 * np.pi/7) ) )

# day
#df4['day_sin'] = df4['day'].apply( lambda x: np.sin( x * (2 * np.pi/30) ) )
#df4['day_cos'] = df4['day'].apply( lambda x: np.cos( x * (2 * np.pi/30) ) )

## 5. FEATURE SELECTION

In [ ]:
df5 = df4.copy()

In [ ]:
# Ordenar por data
df5 = df4.sort_values(by='date').reset_index(drop=True)
df5.head()

### 5.1. Feature Importance

In [ ]:
x_train_n = df5.drop(['casos_de_dengue', 'date'], axis=1)
y_train_n = df5['casos_de_dengue']

In [ ]:
# Model definition
forest = en.ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1)
forest.fit(x_train_n, y_train_n)

In [ ]:
importances = forest.feature_importances_      # Salva as importâncias das features
std = np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
indices = np.argsort(importances)[::-1]        # Importâncias das árvores ordenadas pelo maior valor

#print the feature ranking
print('Feature Ranking')
df = pd.DataFrame()
for i,j in zip(x_train_n,forest.feature_importances_):
    aux = pd.DataFrame({'feature': i, 'importance': j},index=[0])
    df = pd.concat([df,aux],axis=0)
    
print(df.sort_values('importance',ascending=False))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title('Feature Importances')
plt.bar(range(x_train_n.shape[1]), importances[indices], color='g', yerr=std[indices], align='center')
plt.xticks(range(x_train_n.shape[1]),indices)
plt.xlim([-1,x_train_n.shape[1]])
plt.show()

### 5.2. Split dataframe Train / Test

In [ ]:
# Calcula a data de corte
# df5[['amostra', 'date']].groupby('amostra').max().reset_index().max()[1] - datetime.timedelta(days=500)
df5['date'].max() - datetime.timedelta(days=1500)

In [ ]:
# Remove todas as features que foram derivadas
#cols_drop = ['semana_do_ano', 'mes', 'ano_semana']
#df5 =  df5.drop(cols_drop, axis=1)

In [ ]:
# training dataset 
#X_train = df5.iloc[:-1500]
X_train = df5[df5['date'] < '2019-11-22']
y_train = X_train['casos_de_dengue']

# test dataset
#X_test = df5.iloc[-1500:]
X_test = df5[df5['date'] >= '2019-11-22']
y_test = X_test['casos_de_dengue']

print('Training Min Date {}'.format( X_train['date'].min() ))
print('Training Max Date {}'.format( X_train['date'].max() ))

print('\nTest Min Date {}'.format( X_test['date'].min() ))
print('Test Max Date {}'.format( X_test['date'].max() ))

In [ ]:
X_train.shape[0]*100 / df5.shape[0]

In [ ]:
#X_train = X_train.drop(['casos_de_dengue', 'date'], axis=1)
#X_test = X_test.drop(['casos_de_dengue', 'date'], axis=1)
X_train.head()

### 5.3. Manual Feature Selection

In [ ]:
X_train.columns

In [ ]:
#cols_selected = ['presenca_do_mosquito', 'precipitacao', 'umidade', 'temperatura', 'dia',
#       'mes', 'ano', 'semana_do_ano', 'amostra', 'status_conscientizada',
#       'status_desconscientizada', 'status_indisponivel',
#       'status_parcialmente conscientizada']

cols_selected = ['presenca_do_mosquito', 'precipitacao', 'umidade', 'temperatura', 
#                 'ano', 'mes_sin', 'mes_cos',
                 'mes_sin', 'mes_cos'
#                 'status_conscientizada', 'status_desconscientizada', 'status_indisponivel',
#                 'status_parcialmente conscientizada', 
#                 'proliferacao_alta', 'proliferacao_baixa', 'proliferacao_media',
#                  'semana_do_ano_sin', 'semana_do_ano_cos'
                ]

# Adicionar colunas
feat_to_add = ['date', 'casos_de_dengue']

# Features final
cols_selected_full = cols_selected.copy()
cols_selected_full.extend(feat_to_add)

# IV. Modelagem

## 6. MACHINE LEARNING MODELING

In [ ]:
x_train = X_train[cols_selected]
x_test = X_test[cols_selected]

# Time Series Data Preparation
x_training = X_train[cols_selected_full]

### 6.1. Average Model

In [ ]:
#aux1 = x_test.copy()
#aux1['casos_de_dengue'] = y_test.copy()

# prediction
#aux2 = aux1[['amostra', 'casos_de_dengue']].groupby('amostra').mean().reset_index().rename(columns= {'casos_de_dengue': #'predictions'})
#aux1 = pd.merge(aux1, aux2, how='left', on='amostra')
#yhat_baseline = aux1['predictions']

#aux2 = aux1[['amostra', 'casos_de_dengue']].groupby('amostra').mean().reset_index().rename(columns= {'casos_de_dengue': 'predictions'})
#aux1 = pd.merge(aux1, aux2, how='left', on='amostra')
# yhat_baseline = aux1['predictions']

# performance
#baseline_result = ml_error('Average Model', np.expm1( y_test ), np.expm1( yhat_baseline ))
#baseline_result

### 6.2. Linear Regression Model

In [ ]:
# Model
lr = LinearRegression().fit(x_train, y_train)

# Prediction
yhat_lr = lr.predict(x_test)

# Performance
lr_result = ml_error('Linear Regression', np.expm1( y_test ), np.expm1( yhat_lr ))
lr_result

### 6.3. Linear Regression Regularized Model - Lasso

In [ ]:
# Model
lrr = Lasso( alpha= 0.01 ).fit(x_train, y_train)

# Prediction
yhat_lrr = lrr.predict(x_test)

# Performance
lrr_result = ml_error('Linear Regression Regularized - Lasso', np.expm1( y_test ), np.expm1( yhat_lrr ))
lrr_result

### 6.4. Random Forest Regressor

In [ ]:
# Model
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(x_train, y_train)

# Prediction
yhat_rf = rf.predict(x_test)

# Performance
rf_result = ml_error('Random Forest Regressor', np.expm1( y_test ), np.expm1( yhat_rf ))
rf_result

### 7.5. XGBoost Regressor

In [ ]:
# {'n_estimators': 2500, 'eta': 0.03, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 0.7, 'min_child_weight': 15}
# {'n_estimators': 3500, 'eta': 0.03, 'max_depth': 5, 'subsample': 0.5, 'colsample_bytree': 0.3, 'min_child_weight': 15}

# Model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                  n_estimators=2500, 
                                  eta=0.03, 
                                  max_depth=3, 
                                  subsample=0.7,
                                  colsample_bytree=0.7,
                                  min_child_weight=15, random_state=42).fit(x_train, y_train)

# Prediction
yhat_xgb = model_xgb.predict(x_test)

# Performance
xgb_result = ml_error('XGBoost Regressor', np.expm1( y_test ), np.expm1( yhat_xgb ))
xgb_result

# {'n_estimators': 2500, 'eta': 0.03, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 0.7, 'min_child_weight': 15}
# {'n_estimators': 3500, 'eta': 0.03, 'max_depth': 5, 'subsample': 0.5, 'colsample_bytree': 0.3, 'min_child_weight': 15}

# Model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                  n_estimators=3500, 
                                  eta=0.03, 
                                  max_depth=5, 
                                  subsample=0.5,
                                  colsample_bytree=0.3,
                                  min_child_weight=15, random_state=42).fit(x_train, y_train)

# Prediction
yhat_xgb = model_xgb.predict(x_test)

# Performance
xgb_result = ml_error('XGBoost Regressor', np.expm1( y_test ), np.expm1( yhat_xgb ))
xgb_result

### 7.6. Compare Model's Performance

In [ ]:
#modelling_result = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result])
modelling_result = pd.concat([lr_result, lrr_result, rf_result, xgb_result])

modelling_result.sort_values('RMSE').reset_index(drop=True)

## 7. HYPERPARAMETER FINE TUNING

param = {'n_estimators': [1500, 1700, 2500, 3000, 3500],
         'eta': [0.01, 0.03], 
         'max_depth': [3, 5, 9],
         'subsample': [0.1, 0.5, 0.7],
         'colsample_bytree': [0.3, 0.7, 0.9],
         'min_child_weight': [3, 8, 15]}

MAX_EVAL = 5

final_result = pd.DataFrame()

for i in range( MAX_EVAL ):
    # choose values for parameters randomly
    hp = { k: random.sample( v, 1 )[0] for k, v in param.items() }
    print( hp )
    
    # model
    model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                                  n_estimators=hp['n_estimators'], 
                                  eta=hp['eta'], 
                                  max_depth=hp['max_depth'], 
                                  subsample=hp['subsample'],
                                  colsample_bytree=hp['colsample_bytree'],
                                  min_child_weight=hp['min_child_weight'], random_state=42)

    # performance
    result = cross_validation( x_training, 5, 'XGBoost Regressor', model_xgb, verbose=True )
    final_result = pd.concat( [final_result, result] )

final_result

# V. Validação

## 8. ERROR TRANSLATION AND INTERPRETATION

# VI. Implantação

## 9. DEPLOY / SUBMISSION

### 9.1. Deploy

In [ ]:
df_teste = teste.copy()
df_teste.shape

In [ ]:
# 2.2.3. Remover as linhas que não tem presença do mosquito
df_teste = df_teste.drop('casos_de_dengue', axis=1)

# 2.2.4. Converter variáveis
colunas_float = ['precipitacao',
                 'umidade',
                 'temperatura']

colunas_int = ['presenca_do_mosquito']

# Substituir ',' por '.'
for col in colunas_float:
    df_teste[col] = df_teste[col].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

# Converter colunas_float de object para float
for col in colunas_float:
    df_teste[col] = df_teste[col].astype(float)

# Converter date de object para datetime
df_teste['date'] = pd.to_datetime(df_teste['date'])

# 2.4.1. Separar data
# Criar colunas de dia, mes, ano e semana do ano
#df_teste['dia'] = df_teste['date'].dt.day
df_teste['mes'] = df_teste['date'].dt.month
#df_teste['ano'] = df_teste['date'].dt.year
df_teste['semana_do_ano'] = df_teste['date'].dt.weekofyear

# 'ano_semana': semana do ano, no formato string 'YYYY-WW'
#df_teste['ano_semana'] = df_teste['date'].dt.strftime( '%Y-%W' )

# 2.2.4. Preencher NA's
presenca_mediana = df_teste['presenca_do_mosquito'].median()
df_teste['presenca_do_mosquito'] = df_teste['presenca_do_mosquito'].fillna(presenca_mediana)

preencher_media_semana(df2, ['umidade', 'temperatura', 'precipitacao'])

# Converter colunas_int de object para float
for col in colunas_int:
    df_teste[col] = df_teste[col].astype(int)

# 2.2.5. Excluir os demais NA's
df_teste = df_teste.fillna(0)
#df_teste = df_teste.dropna().reset_index(drop=True)
df_teste['casos_de_dengue'] = np.nan

# 2.4. FEATURE ENGINEERING

# 2.4.2. Separar em 3 grupos
#df_teste['proliferacao'] = df_teste['semana_do_ano'].apply(lambda x: definir_proliferacao(x))

# 4.1. Normalização do fenômeno
df_teste['casos_de_dengue'] = np.log1p(df_teste['casos_de_dengue'])

# 4.2. Rescala de Variáveis Numéricas
mm = pp.MinMaxScaler()
ss = pp.StandardScaler()
rs = pp.RobustScaler()

df_teste['presenca_do_mosquito'] = mm.fit_transform(df_teste[['presenca_do_mosquito']])
df_teste['precipitacao']         = ss.fit_transform(df_teste[['precipitacao']])
df_teste['umidade']              = ss.fit_transform(df_teste[['umidade']])
df_teste['temperatura']          = ss.fit_transform(df_teste[['temperatura']])

# 4.3. Encodar variáveis categóricas
#df_teste = pd.get_dummies(df_teste, prefix=['status'], columns=['status'])
#df_teste = pd.get_dummies(df_teste, prefix=['proliferacao'], columns=['proliferacao'])

# 4.4. Transformação de natureza
# mes
df_teste['mes_sin'] = df_teste['mes'].apply( lambda x: np.sin( x * (2 * np.pi/12) ) )
df_teste['mes_cos'] = df_teste['mes'].apply( lambda x: np.cos( x * (2 * np.pi/12) ) )

# week_of_year
df_teste['semana_do_ano_sin'] = df_teste['semana_do_ano'].apply( lambda x: np.sin( x * (2 * np.pi/52) ) )
df_teste['semana_do_ano_cos'] = df_teste['semana_do_ano'].apply( lambda x: np.cos( x * (2 * np.pi/52) ) )

In [ ]:
df_teste.shape

### 9.2. Submission

In [ ]:
df_teste.columns

In [ ]:
aux_teste  = df_teste[cols_selected]
y_hat_test = model_xgb.predict(aux_teste)

In [ ]:
final_test = pd.concat([df_teste, pd.Series(np.expm1(y_hat_test), name='Casos de dengue')], axis=1)
final_test = final_test[['date', 'Casos de dengue']].sort_values('date')

In [ ]:
final_test.shape

In [ ]:
final_test = final_test.rename(columns={'date': 'Date', 'Casos de dengue': 'Casos de dengue'})
final_test

In [ ]:
# Exportar o arquivo
final_test.to_csv('datasets/submission_xgb_XX.csv', index=False)

In [ ]:
# 48065.99724